In [22]:
# This script analyzes .mat files generated LickToGetReward_Training or LickToGetReward
# to get "mean_num_licksperbout", "mean_num_multi_licks", "mean_bout_duration", 
# "mean_frequency", "Avg. Licks in Dringking period ", "LickThreshold"
# and save them in csv file

# Import libraries
import os
import numpy as np
from scipy.io import loadmat
import glob
import csv

# Set the working directory to the folder containing the .mat files
os.chdir('/Users/jun/Documents/Work/Project/Intralingual parasympathetic ganglion neurons/Behavior/ILPG_Silencing_Di_#3/')
dir_path = os.getcwd()

# Use glob to select all .mat files in the directory except those containing "adjusted_data" in the file name
file_list = glob.glob(dir_path + '/*.mat')
mat_files = [f for f in file_list if 'adjusted_data' not in f]

# Define column names
column_names = ["mat_file", "mean_num_licksperbout", "mean_num_multi_licks", "mean_bout_duration", "mean_frequency", "Avg. Licks in Dringking period ", "LickThreshold"]

# Check if file exists
file_exists = os.path.isfile("results.csv")

# Write the column names only if the file does not exist
if not file_exists:
    with open("results2.csv", "w", newline="") as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(column_names)

# Use glob to select all .mat files in the directory except those containing "adjusted_data" in the file name
file_list = glob.glob(dir_path + '/*.mat')
mat_files = [f for f in file_list if 'adjusted_data' not in f]

# Iterate over the .mat files
for mat_file in mat_files:
    # Load the original data
    mat_contents = loadmat(mat_file)
    SessionData = mat_contents['SessionData']

    n_trials = SessionData['nTrials'][0, 0][0, 0]
    raw_events = SessionData['RawEvents'][0, 0]['Trial'][0, 0]
    
    # Extract lickometer data
    lickometer = {
        'BNCHigh': [np.nan] * n_trials,
        'BNCLow': [np.nan] * n_trials
    }

    for i in range(n_trials):
        trial = raw_events[0, i]['Events'][0, 0]

        if 'BNC1High' not in trial.dtype.names or 'BNC1Low' not in trial.dtype.names:
            lickometer['BNCHigh'][i] = np.nan
            lickometer['BNCLow'][i] = np.nan
        else:
            lickometer['BNCHigh'][i] = trial['BNC1High'][0, 0]
            lickometer['BNCLow'][i] = trial['BNC1Low'][0, 0]

    trials = SessionData["RawEvents"][0, 0]["Trial"]
    num_trials = SessionData['nTrials'][0, 0][0, 0]

    # Initialize variables
    concatenated_BNC1High = []
    cumulative_trial_duration = 0.0

    for trial in range(num_trials):
        # Get state information for current trial
        state_info = None
        if trial < len(trials.item()[0]):
            state_info = trials.item()[0, trial]["States"][0, 0]

        # Calculate trial duration
        trial_duration = 0
        if state_info is not None:
            if "Drinking" in state_info.dtype.fields.keys() and not np.isnan(state_info["Drinking"][0][0]).all():
                trial_end_time = state_info["Drinking"][0][0][0][1]
            elif "WaitForLick" in state_info.dtype.fields.keys() and not np.isnan(state_info["WaitForLick"][0][0]).all():
                trial_end_time = state_info["WaitForLick"][0][0][0][1]
            trial_duration = trial_end_time

        # Add trial duration to the cumulative_trial_duration
        if trial == 0:
            cumulative_trial_duration = trial_end_time
        else:
            cumulative_trial_duration += trial_duration

        # Concatenate BNC1High values
        if not np.isnan(lickometer["BNCHigh"][trial]).all():
            trial_BNC1High = lickometer["BNCHigh"][trial][0] + cumulative_trial_duration - trial_duration
            concatenated_BNC1High.extend(trial_BNC1High)

        
    licks = concatenated_BNC1High

    # find lick bout
    bouts = []
    currentbout = []

    # Iterate over the elements of the licks list
    for i in range(len(licks)):
        # Add the current data point to the current aggregate
        currentbout.append(licks[i])

        # If the gap between the current data point and the next one is greater than 0.25 (4Hz),
        # store the current aggregate and start a new one 
        if i < len(licks) - 1 and licks[i+1] - licks[i] > 0.25:
            bouts.append(currentbout)
            currentbout = []

    # Store the last aggregate
    if currentbout:
        bouts.append(currentbout)

    # count number of licks in one lick bout
    num_licks = [len(bout) for bout in bouts]
    mean_num_licksperbout = np.mean(num_licks)

    # exclude single licks
    multi_licks = [nlicks for nlicks in num_licks if nlicks != 1]
    mean_num_multi_licks = np.mean(multi_licks)

    # mean lick bout duration
    # Find bouts with more than one data point
    bouts = [b for b in bouts if len(b) > 1]

    # Calculate differences for each bout
    differences = [b[-1] - b[0] for b in bouts]

    # Calculate the mean bout duration
    mean_bout_duration = sum(differences) / len(differences)

    # Frequency
    mean_frequencies = []
    for bout in bouts:
        if len(bout) > 5:
            bout_diff = np.diff(bout)
            # create a mask to select elements larger than or equal to 0.08. Max frequency allowed is 12Hz
            mask = bout_diff >= 0.08
            bout_diff = bout_diff[mask]
            bout_freq = 1 / bout_diff
            mean_frequency = np.mean(bout_freq)
            mean_frequencies.append(mean_frequency)

    mean_frequency = np.mean(mean_frequencies)


    total_drinking_duration = 0.0
    total_BNC1High_count = 0.0

    trials = SessionData["RawEvents"][0, 0]["Trial"]
    num_trials = SessionData['nTrials'][0, 0][0, 0]

    for trial in range(num_trials):
        state_info = None
        if trial < len(trials.item()[0]):
            state_info = trials.item()[0, trial]["States"][0, 0]

        if state_info is not None and "Drinking" in state_info.dtype.fields.keys():
            drinking_start = state_info["Drinking"][0][0][0][0]
            drinking_end = state_info["Drinking"][0][0][0][1]

            if not (np.isnan(drinking_start) or np.isnan(drinking_end)):
                drinking_duration = drinking_end - drinking_start
                total_drinking_duration += drinking_duration
                # print(f"Trial: {trial}, drinking_duration: {drinking_duration}, total_drinking_duration: {total_drinking_duration}")

                if not np.isnan(lickometer["BNCHigh"][trial]).all():
                    BNC1High_count = sum([1 for l in lickometer["BNCHigh"][trial][0] if l >= drinking_start and l <= drinking_end])
                else:
                    BNC1High_count = 0
                total_BNC1High_count += BNC1High_count  


    average_BNC1High_count = total_BNC1High_count / num_trials if num_trials > 0 else np.nan
    average_frequency = (total_BNC1High_count / total_drinking_duration) if total_drinking_duration > 0 else np.nan

    # Get LickThreshold
    LickThreshold = SessionData['TrialSettings'][0, 0][0, 0]['GUI'][0, 0]['LickThreshold'][0, 0]

    # At the end of the processing, append the results to the CSV file
    # Check if file exists
    file_exists = os.path.isfile("results.csv")

    # Define column names
    column_names = ["mat_file", "mean_num_licksperbout", "mean_num_multi_licks", "mean_bout_duration", "mean_frequency", "Avg. Licks in Dringking period ", "LickThreshold"]


   # Open the CSV file in append mode
    with open("results2.csv", "a", newline="") as csvfile:
        csv_writer = csv.writer(csvfile)

        mat_file_name = os.path.basename(mat_file)

        # Write the data row
        csv_writer.writerow([mat_file_name, mean_num_licksperbout, mean_num_multi_licks, mean_bout_duration, np.mean(mean_frequencies), average_BNC1High_count, LickThreshold])

# Import libraries
import os
import numpy as np
from scipy.io import loadmat
import glob
import csv

# Set the working directory to the folder containing the .mat files
os.chdir('/Users/jun/Documents/Work/Project/Intralingual parasympathetic ganglion neurons/Behavior/ILPG_Silencing_Di_#3/')
dir_path = os.getcwd()

# Use glob to select all .mat files in the directory except those containing "adjusted_data" in the file name
file_list = glob.glob(dir_path + '/*.mat')
mat_files = [f for f in file_list if 'adjusted_data' not in f]

# Define column names
column_names = ["mat_file", "mean_num_licksperbout", "mean_num_multi_licks", "mean_bout_duration", "mean_frequency", "Avg. Licks in Dringking period ", "LickThreshold"]

# Check if file exists
file_exists = os.path.isfile("results.csv")

# Write the column names only if the file does not exist
if not file_exists:
    with open("results2.csv", "w", newline="") as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(column_names)

# Use glob to select all .mat files in the directory except those containing "adjusted_data" in the file name
file_list = glob.glob(dir_path + '/*.mat')
mat_files = [f for f in file_list if 'adjusted_data' not in f]

# Iterate over the .mat files
for mat_file in mat_files:
    # Load the original data
    mat_contents = loadmat(mat_file)
    SessionData = mat_contents['SessionData']

    n_trials = SessionData['nTrials'][0, 0][0, 0]
    raw_events = SessionData['RawEvents'][0, 0]['Trial'][0, 0]
    
    # Extract lickometer data
    lickometer = {
        'BNCHigh': [np.nan] * n_trials,
        'BNCLow': [np.nan] * n_trials
    }

    for i in range(n_trials):
        trial = raw_events[0, i]['Events'][0, 0]

        if 'BNC1High' not in trial.dtype.names or 'BNC1Low' not in trial.dtype.names:
            lickometer['BNCHigh'][i] = np.nan
            lickometer['BNCLow'][i] = np.nan
        else:
            lickometer['BNCHigh'][i] = trial['BNC1High'][0, 0]
            lickometer['BNCLow'][i] = trial['BNC1Low'][0, 0]

    trials = SessionData["RawEvents"][0, 0]["Trial"]
    num_trials = SessionData['nTrials'][0, 0][0, 0]

    # Initialize variables
    concatenated_BNC1High = []
    cumulative_trial_duration = 0.0

    for trial in range(num_trials):
        # Get state information for current trial
        state_info = None
        if trial < len(trials.item()[0]):
            state_info = trials.item()[0, trial]["States"][0, 0]

        # Calculate trial duration
        trial_duration = 0
        if state_info is not None:
            if "Drinking" in state_info.dtype.fields.keys() and not np.isnan(state_info["Drinking"][0][0]).all():
                trial_end_time = state_info["Drinking"][0][0][0][1]
            elif "WaitForLick" in state_info.dtype.fields.keys() and not np.isnan(state_info["WaitForLick"][0][0]).all():
                trial_end_time = state_info["WaitForLick"][0][0][0][1]
            trial_duration = trial_end_time

        # Add trial duration to the cumulative_trial_duration
        if trial == 0:
            cumulative_trial_duration = trial_end_time
        else:
            cumulative_trial_duration += trial_duration

        # Concatenate BNC1High values
        if not np.isnan(lickometer["BNCHigh"][trial]).all():
            trial_BNC1High = lickometer["BNCHigh"][trial][0] + cumulative_trial_duration - trial_duration
            concatenated_BNC1High.extend(trial_BNC1High)

        
    licks = concatenated_BNC1High

    # find lick bout
    bouts = []
    currentbout = []

    # Iterate over the elements of the licks list
    for i in range(len(licks)):
        # Add the current data point to the current aggregate
        currentbout.append(licks[i])

        # If the gap between the current data point and the next one is greater than 0.25 (4Hz),
        # store the current aggregate and start a new one 
        if i < len(licks) - 1 and licks[i+1] - licks[i] > 0.25:
            bouts.append(currentbout)
            currentbout = []

    # Store the last aggregate
    if currentbout:
        bouts.append(currentbout)

    # count number of licks in one lick bout
    num_licks = [len(bout) for bout in bouts]
    mean_num_licksperbout = np.mean(num_licks)

    # exclude single licks
    multi_licks = [nlicks for nlicks in num_licks if nlicks != 1]
    mean_num_multi_licks = np.mean(multi_licks)

    # mean lick bout duration
    # Find bouts with more than one data point
    bouts = [b for b in bouts if len(b) > 1]

    # Calculate differences for each bout
    differences = [b[-1] - b[0] for b in bouts]

    # Calculate the mean bout duration
    mean_bout_duration = sum(differences) / len(differences)

    # Frequency
    mean_frequencies = []
    for bout in bouts:
        if len(bout) > 5:
            bout_diff = np.diff(bout)
            # create a mask to select elements larger than or equal to 0.08. Max frequency allowed is 12Hz
            mask = bout_diff >= 0.08
            bout_diff = bout_diff[mask]
            bout_freq = 1 / bout_diff
            mean_frequency = np.mean(bout_freq)
            mean_frequencies.append(mean_frequency)

    mean_frequency = np.mean(mean_frequencies)


    total_drinking_duration = 0.0
    total_BNC1High_count = 0.0

    trials = SessionData["RawEvents"][0, 0]["Trial"]
    num_trials = SessionData['nTrials'][0, 0][0, 0]

    for trial in range(num_trials):
        state_info = None
        if trial < len(trials.item()[0]):
            state_info = trials.item()[0, trial]["States"][0, 0]

        if state_info is not None and "Drinking" in state_info.dtype.fields.keys():
            drinking_start = state_info["Drinking"][0][0][0][0]
            drinking_end = state_info["Drinking"][0][0][0][1]

            if not (np.isnan(drinking_start) or np.isnan(drinking_end)):
                drinking_duration = drinking_end - drinking_start
                total_drinking_duration += drinking_duration
                # print(f"Trial: {trial}, drinking_duration: {drinking_duration}, total_drinking_duration: {total_drinking_duration}")

                if not np.isnan(lickometer["BNCHigh"][trial]).all():
                    BNC1High_count = sum([1 for l in lickometer["BNCHigh"][trial][0] if l >= drinking_start and l <= drinking_end])
                else:
                    BNC1High_count = 0
                total_BNC1High_count += BNC1High_count  


    average_BNC1High_count = total_BNC1High_count / num_trials if num_trials > 0 else np.nan
    average_frequency = (total_BNC1High_count / total_drinking_duration) if total_drinking_duration > 0 else np.nan

    # Get LickThreshold
    LickThreshold = SessionData['TrialSettings'][0, 0][0, 0]['GUI'][0, 0]['LickThreshold'][0, 0]

    # At the end of the processing, append the results to the CSV file
    # Check if file exists
    file_exists = os.path.isfile("results.csv")

    # Define column names
    column_names = ["mat_file", "mean_num_licksperbout", "mean_num_multi_licks", "mean_bout_duration", "mean_frequency", "Avg. Licks in Dringking period ", "LickThreshold"]


   # Open the CSV file in append mode
    with open("results2.csv", "a", newline="") as csvfile:
        csv_writer = csv.writer(csvfile)

        mat_file_name = os.path.basename(mat_file)

        # Write the data row
        csv_writer.writerow([mat_file_name, mean_num_licksperbout, mean_num_multi_licks, mean_bout_duration, np.mean(mean_frequencies), average_BNC1High_count, LickThreshold])

